# Cleaning Data Walk Through

This walkthrough is an example of cleaning data, in this case to be used for modelling. Here, the training data for a model includes two columns of information that the test data does not, so these need to be added on from another source.

To be able to run this exact code, you will need your file structure set up with a _data_ folder inside your working directory. _analysis_ and _output_ folders are also recommended to keep your files tidy, but you can follow this walkthrough without. The data you will need can be found on the [Github repository for this Hackathon](https://github.com/MangoTheCat/rss-2018-hackathon). 


To begin with we will import the modules need during this walkthrough. You may need to install first.


In [3]:
import pandas as pd

## Investigating the Data

The data we are looking at is election data from the 2015 and 2017 general elections, where a model will be trained on 2015 results and tested on 2017 results. The data sets available to us are `ge_2015_2017_prediction_data.csv` and `HoC-GE2017-constituency-results.csv`, both of which you should **download** from the [Github repository](https://github.com/MangoTheCat/rss-2018-hackathon) into your _data_ folder. We first upload these files as dataframes.

In [9]:
pred_data = pd.read_csv("data/ge_2015_2017_prediction_data.csv")
actual_data = pd.read_csv("data/HoC-GE2017-constituency-results.csv")

The first file has everything that is needed for the model, except the name of the winning party and the percentage of the vote secured by that party. These can be found from the second file, and so we need to find a way of combining these two files.


Let's look at the column names to see if there are any common columns.


In [11]:
pred_data.columns

Index(['Press Association ID Number', 'Constituency Name', 'Region', 'geo',
       'Electorate', 'Votes', 'party', 'votes_last', 'pc_last', 'win_last',
       'polls_now', 'swing_now', 'swing_forecast_pc', 'swing_forecast_win',
       'con', 'grn', 'lab', 'ld', 'pc', 'snp', 'ukip', 'Region_East',
       'Region_East Midlands', 'Region_London', 'Region_North East',
       'Region_North West', 'Region_Northern Ireland', 'Region_Scotland',
       'Region_South East', 'Region_South West', 'Region_Wales',
       'Region_West Midlands', 'Region_Yorkshire and The Humber',
       'won_here_last'],
      dtype='object')

In [14]:
actual_data.columns

Index(['ons_id', 'ons_region_id', 'constituency_name', 'county_name',
       'region_name', 'country_name', 'constituency_type', 'declaration_time',
       'result', 'first_party', 'second_party', 'electorate', 'valid_votes',
       'invalid_votes', 'majority', 'con', 'lab', 'ld', 'ukip', 'green', 'snp',
       'pc', 'dup', 'sf', 'sdlp', 'uup', 'alliance', 'other', 'other_winner'],
      dtype='object')

From looking at these column names, we can see that both include a constituency name and some sort of ID. First we look at the constituency names to see if they match.

In [23]:
len(set(pred_data['Constituency Name']) - set(actual_data['constituency_name']))

264

There are a **lot** of mismatches in the names, and fixing all the strings to match will be a lot of work, so let's look at the ID fields instead.

In [29]:
pred_data['Press Association ID Number'].unique()[:6]

array([1., 2., 3., 4., 5., 6.])

In [30]:
actual_data['ons_id'].unique()[:6]

array(['W07000049', 'W07000058', 'S14000001', 'S14000002', 'S14000003',
       'E14000530'], dtype=object)

In [32]:
len(pred_data['Press Association ID Number'].unique())

650

In [34]:
len(actual_data['ons_id'].unique())

650

We can see that although the two types of IDs do not match, there are the same number of unique IDs of both types, meaning that it should be possible to match them. Luckily, the `ge_2015_results.csv` file available on the [Github repository](https://github.com/MangoTheCat/rss-2018-hackathon), contains both types of ID and so we can use it for matching. You should **download** this file into your _data_ folder before continuing. Now we will upload this file, and look at the first few column names.


## Matching IDs

In [42]:
ID_match = pd.read_csv("data/ge_2015_results.csv")
ID_match.columns[:10]

Index(['Press Association ID Number', 'Constituency ID', 'Constituency Name',
       'Constituency Type', 'County', 'Region ID', 'Region', 'Country',
       'Election Year', 'Electorate'],
      dtype='object')

We assume the `Press.Association.ID.Number` columns match, and if we look at `Constituency.ID` we can see it matches `ons_id`.

In [44]:
ID_match['Constituency ID'][:5]

0    W07000049
1    W07000058
2    S14000001
3    S14000002
4    S14000058
Name: Constituency ID, dtype: object

In [47]:
set(actual_data['ons_id']) == set(ID_match['Constituency ID'])

True

As we only need these two ID columns to give us a reference table, we delete all other columns and rename `Constituency.ID` to `ons_id`, so it can be merged later. We will then check the table is as expected.

In [65]:
ID_match = ID_match.iloc[:,0:2]
ID_match = ID_match.rename(columns = {"Constituency ID": "ons_id"})
ID_match.head()

,Press Association ID Number,ons_id
0,1.0,W07000049
1,2.0,W07000058
2,3.0,S14000001
3,4.0,S14000002
4,5.0,S14000058


Now that we have a reference table linking the two ID numbers together, we can merge this with the `actual_data` dataframe by the `ons_id` column, so that the `actual_data` contains `Press.Association.ID.Number`. We can then check that this extra column has indeed been added correctly by looking at column names and checking that every constituency now has a `Press.Association.ID.Number`.


In [69]:
actual_data = pd.merge(actual_data, ID_match)
actual_data.columns

Index(['ons_id', 'ons_region_id', 'constituency_name', 'county_name',
       'region_name', 'country_name', 'constituency_type', 'declaration_time',
       'result', 'first_party', 'second_party', 'electorate', 'valid_votes',
       'invalid_votes', 'majority', 'con', 'lab', 'ld', 'ukip', 'green', 'snp',
       'pc', 'dup', 'sf', 'sdlp', 'uup', 'alliance', 'other', 'other_winner',
       'Press Association ID Number'],
      dtype='object')

## Selecting Useful Data

We are now at the point where we have a matching ID column in our `pred_data` and `actual_data`, but as we do not need all the columns from `actual_data`, we are going to pick out and calculate what we need before merging. The data missing from the `pred_data` is the winning party and the percentage of valid votes that went to that party. The first is already present in the `actual_data` as `first_party`, but the second will have to be calculated using `valid_votes` and `majority`. This means we can select only these columns into a dataframe and calculate the percentage, then drop the columns used only for percentage calculations. We will also name the columns to match the column names used in the model.

In [86]:
actual_cols = ['Press Association ID Number', 'first_party', 'valid_votes', 'majority']
need_actual_data = actual_data[actual_cols]

need_actual_data = need_actual_data.rename(columns = {"first_party": "actual_win_now"})
print(need_actual_data.columns)
need_actual_data['actual_pc_now'] = need_actual_data['majority'] / need_actual_data['valid_votes']

selected_cols = ['Press Association ID Number', 'actual_win_now', 'actual_pc_now']
need_actual_data = need_actual_data[selected_cols]

Index(['Press Association ID Number', 'actual_win_now', 'valid_votes',
       'majority'],
      dtype='object')


Before continuing, let's check that this dataframe is exactly as we expect.


In [89]:
need_actual_data.shape

(650, 3)

In [90]:
need_actual_data.head()

,Press Association ID Number,actual_win_now,actual_pc_now
0,1.0,Lab,0.503817
1,2.0,Con,0.019751
2,3.0,SNP,0.112604
3,4.0,Con,0.106803
4,6.0,SNP,0.005131


## Merging the DataFrames

Now that we have the extra data we need, and an ID column that matches to `pred_data`, we can merge the two to give our full data set.


In [94]:
full_data = pd.merge(pred_data, need_actual_data)

# Check the dimensions
print(full_data.shape)

# Check there are no missing values (should be False!)
full_data.isnull().values.any()

(4550, 36)


False

## Saving the File

We can see that the merge worked - we have two extra columns, the same number of rows, and no NA values, showing us that we have lost no data during this process. As a final step we will save this dataframe as a new csv file.

In [97]:
full_data.to_csv("data/ge_2017_results.csv")